### Emotiefflib 이용 감정 분석

In [ ]:
from typing import List
import cv2
import matplotlib.pyplot as plt
import numpy as np
from facenet_pytorch import MTCNN
from emotiefflib.facial_analysis import EmotiEffLibRecognizer, get_model_list
def recognize_faces(frame: np.ndarray, device: str) -> List[np.ndarray]:
    def detect_face(frame: np.ndarray):
        mtcnn = MTCNN(keep_all=False, post_process=False, min_face_size=40, device=device)
        bounding_boxes, probs = mtcnn.detect(frame, landmarks=False)
        if probs[0] is None:
            return []
        bounding_boxes = bounding_boxes[probs > 0.9]
        return bounding_boxes

    bounding_boxes = detect_face(frame)
    facial_images = []
    for bbox in bounding_boxes:
        box = bbox.astype(int)
        x1, y1, x2, y2 = box[0:4]
        facial_images.append(frame[y1:y2, x1:x2, :])
    return facial_images

In [45]:
def emotion(image_path:str):
    emotion_class = ['분노','경멸','혐오','두려움','행복','보통','슬픔','놀람']
    device = "cpu"
    model_name = get_model_list()[0]
    frame_bgr = cv2.imread(image_path)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    facial_images = recognize_faces(frame, device)
    fer = EmotiEffLibRecognizer(engine="onnx", model_name=model_name, device=device)

    emotions = []
    for face_img in facial_images:
        emotion, test = fer.predict_emotions(face_img, logits=True)
        emotions.append(test[0].tolist())
        # print(emotion,test)
    return emotions, emotion_class
def softmax(lst):
    min_val = min(lst)
    max_val = max(lst)
    if max_val == min_val:
        return [0.0 for _ in lst]  # 모든 값이 같을 경우 0으로 처리
    return [(x - min_val) / (max_val - min_val) for x in lst]

In [37]:
image = './datasets/images/4001.jpg'
emo, cls = emotion(image)

In [39]:
emo[0]

[1.2283426523208618,
 -2.95230770111084,
 0.41483330726623535,
 3.835360288619995,
 -5.456241130828857,
 0.8360392451286316,
 5.4043192863464355,
 0.5669528245925903]

In [40]:
cls

['분노', '경멸', '혐오', '두려움', '행복', '보통', '슬픔', '놀람']

In [43]:
def softmax(lst):
    min_val = min(lst)
    max_val = max(lst)
    if max_val == min_val:
        return [0.0 for _ in lst]  # 모든 값이 같을 경우 0으로 처리
    return [(x - min_val) / (max_val - min_val) for x in lst]

In [44]:
softmax(emo[0])

[0.6154916069136239,
 0.23055287513139786,
 0.5405866928202303,
 0.8555360922954556,
 0.0,
 0.5793697686177082,
 1.0,
 0.5545932920640214]